https://www.transtats.bts.gov/Fields.asp?Table_ID=236

In [35]:
from __future__ import division #, print_function # Imports from __future__ since we're running Python 2

In [36]:
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [37]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
random_state = 0
%matplotlib inline
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from datetime import datetime

from helpers.feature_engineering import dateStrToDayYear
from helpers.my_one_hot_encoder import MyOneHotEncoder

In [38]:
rng = np.random.RandomState(seed=random_state)

# Flying to New York City - Target Classes Preprocessing

In [39]:
path_data = os.path.realpath(os.path.join(os.getcwd(), '../Data', 'binary_train_data.csv'))
assert os.path.isfile(path_data)
path_data

'/home/student/pligor.george@gmail.com/msc_Artificial_Intelligence/dme_Data_Mining/dmedatarats/Data/binary_train_data.csv'

In [40]:
df = pd.read_csv(path_data, delimiter = ',', header=0)
df.shape

(186308, 40)

### Feature Date

In [41]:
df[:5]

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,...,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,IS_DELAYED
0,2016,3,7,3,7,2016-07-03,HA,19690,HA,50,...,0.0,0.0,-2.0,0.0,585.0,574.0,1.0,4983.0,11,False
1,2016,2,4,2,6,2016-04-02,B6,20409,B6,1304,...,8.0,0.0,0.0,0.0,246.0,264.0,1.0,1598.0,7,False
2,2016,1,3,26,6,2016-03-26,B6,20409,B6,164,...,43.0,1.0,2.0,0.0,164.0,147.0,1.0,1041.0,5,True
3,2016,1,1,13,3,2016-01-13,B6,20409,B6,429,...,0.0,0.0,-1.0,0.0,193.0,193.0,1.0,1074.0,5,False
4,2016,4,12,18,7,2016-12-18,DL,19790,DL,786,...,0.0,0.0,-1.0,0.0,193.0,184.0,1.0,1076.0,5,False


The year attribute is currently the same year, so we could just remove it.
While the flight date attribute brings little information extra information. It could be more useful to convert it to the day of the year since some days could have some particular on a yearly basis.
Although we understand that this attribute will have a correlation with the Quarter, Month, and Day of Month attributes.

In [42]:
df['FL_DATE'].values.shape#[np.newaxis].shape

(186308,)

In [43]:
df['YDAY'] = [dateStrToDayYear(dateStr) for dateStr in df['FL_DATE']]

In [44]:
df[:5]

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,...,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,IS_DELAYED,YDAY
0,2016,3,7,3,7,2016-07-03,HA,19690,HA,50,...,0.0,-2.0,0.0,585.0,574.0,1.0,4983.0,11,False,185
1,2016,2,4,2,6,2016-04-02,B6,20409,B6,1304,...,0.0,0.0,0.0,246.0,264.0,1.0,1598.0,7,False,93
2,2016,1,3,26,6,2016-03-26,B6,20409,B6,164,...,1.0,2.0,0.0,164.0,147.0,1.0,1041.0,5,True,86
3,2016,1,1,13,3,2016-01-13,B6,20409,B6,429,...,0.0,-1.0,0.0,193.0,193.0,1.0,1074.0,5,False,13
4,2016,4,12,18,7,2016-12-18,DL,19790,DL,786,...,0.0,-1.0,0.0,193.0,184.0,1.0,1076.0,5,False,353


In [45]:
df_1 = df.drop(labels=['YEAR', 'FL_DATE'], axis=1)
df_1.shape

(186308, 39)

In [46]:
df_1[:5]

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,...,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,IS_DELAYED,YDAY
0,3,7,3,7,HA,19690,HA,50,12173,HNL,...,0.0,-2.0,0.0,585.0,574.0,1.0,4983.0,11,False,185
1,2,4,2,6,B6,20409,B6,1304,14843,SJU,...,0.0,0.0,0.0,246.0,264.0,1.0,1598.0,7,False,93
2,1,3,26,6,B6,20409,B6,164,14986,SRQ,...,1.0,2.0,0.0,164.0,147.0,1.0,1041.0,5,True,86
3,1,1,13,3,B6,20409,B6,429,12478,JFK,...,0.0,-1.0,0.0,193.0,193.0,1.0,1074.0,5,False,13
4,4,12,18,7,DL,19790,DL,786,12953,LGA,...,0.0,-1.0,0.0,193.0,184.0,1.0,1076.0,5,False,353


### Feature Unique Carrier

In [47]:
carriers = np.unique(df_1['UNIQUE_CARRIER'])
carriers

array(['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'NK', 'OO', 'UA', 'VX',
       'WN'], dtype=object)

In [48]:
len(carriers)

12

We see that this feature is categorical with a finite number of carriers which they could be represented with multiple features using one-hot-encoding

In [51]:
unique_carrier_one_hot = MyOneHotEncoder().encodePandasColumn(data_frame = df_1, col_name = 'UNIQUE_CARRIER')

In [54]:
df_2 = pd.concat((df_1, unique_carrier_one_hot), axis=1).drop(labels=['UNIQUE_CARRIER'], axis=1)
df_2.shape

(186308, 49)

In [55]:
df_2[:5]

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,AIRLINE_ID,CARRIER,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,...,UNIQUE_CARRIER_1,UNIQUE_CARRIER_2,UNIQUE_CARRIER_3,UNIQUE_CARRIER_4,UNIQUE_CARRIER_5,UNIQUE_CARRIER_6,UNIQUE_CARRIER_7,UNIQUE_CARRIER_8,UNIQUE_CARRIER_9,UNIQUE_CARRIER_10
0,3,7,3,7,19690,HA,50,12173,HNL,"Honolulu, HI",...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2,4,2,6,20409,B6,1304,14843,SJU,"San Juan, PR",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,26,6,20409,B6,164,14986,SRQ,"Sarasota/Bradenton, FL",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,13,3,20409,B6,429,12478,JFK,"New York, NY",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,12,18,7,19790,DL,786,12953,LGA,"New York, NY",...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Also from previous analysis we have established that carrier is redundant. Let's confirm it here and remove this column

In [57]:
assert set(df['UNIQUE_CARRIER']) == set(df['CARRIER'])

In [59]:
df_3 = df_2.drop(labels=['CARRIER'], axis=1)
df_3.shape

(186308, 48)

In [60]:
df_3[:5]

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,AIRLINE_ID,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,...,UNIQUE_CARRIER_1,UNIQUE_CARRIER_2,UNIQUE_CARRIER_3,UNIQUE_CARRIER_4,UNIQUE_CARRIER_5,UNIQUE_CARRIER_6,UNIQUE_CARRIER_7,UNIQUE_CARRIER_8,UNIQUE_CARRIER_9,UNIQUE_CARRIER_10
0,3,7,3,7,19690,50,12173,HNL,"Honolulu, HI",HI,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2,4,2,6,20409,1304,14843,SJU,"San Juan, PR",PR,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,26,6,20409,164,14986,SRQ,"Sarasota/Bradenton, FL",FL,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,13,3,20409,429,12478,JFK,"New York, NY",NY,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,12,18,7,19790,786,12953,LGA,"New York, NY",NY,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Binary classification and undersampling

Let's start by making it as easy as possible by undersampling the non-delayed class and keeping only a number of non-delayed instances that are equal to the delayed ones making it a balanced binary classification problem

In [31]:
nondelayed = df[df['ARR_DELAY_GROUP'] <= 0]
nondelayed.shape

(340341, 39)

In [34]:
delayed = df[df['ARR_DELAY_GROUP'] > 0]
delayed.shape

(93154, 39)

In [45]:
non_delayed_short = shuffle(nondelayed, random_state = random_state, n_samples = len(delayed))
non_delayed_short.shape

(93154, 39)

In [46]:
delayed['IS_DELAYED'] = True

/home/student/anaconda2/envs/dme/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [47]:
non_delayed_short['IS_DELAYED'] = False

/home/student/anaconda2/envs/dme/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [58]:
flights = shuffle(pd.concat((delayed, non_delayed_short)), random_state = random_state)
flights.shape

(186308, 40)

In [59]:
flights.to_csv(
    os.path.realpath(os.path.join(os.getcwd(), '../Data', 'binary_train_data.csv')),
    index=False)